In [25]:
import pandas as pd
import numpy as np
filename = 'nba_elo_latest.csv'
df = pd.read_csv(filename)
df['game_id']=range(1,len(df)+1) #Create a new column for Game IDs
df=df[['game_id','home','away','elo1_pre_home','elo2_pre_away','home_score','away_score']]


df['win']=np.where(df['home_score']>df['away_score'],1,0) #Create a new column for game result
df['diff']=np.where((df['home_score']-df['away_score'])<0,-1*(df['home_score']-df['away_score']),df['home_score']-df['away_score']) #Create a new column for modulus of the difference between the scores
df['d']=np.where(df['win']==1,df['elo1_pre_home']-df['elo2_pre_away']+100,df['elo2_pre_away']-df['elo1_pre_home']-100) #Create a new column to find the difference between the ELO Ratings of the 2 teams inclusive of the home team advantage
df['mov']=(df['diff']+3)**0.8/(7.5+(0.006*df['d'])) #Create a new column for the Margin of Victory
df['K']=df['mov']*20 #Created a new column for the adjusted K factor that includes the Margin of Victory
df['elo_prob_home']=1/(1+10**((df['elo2_pre_away']-df['elo1_pre_home']-100)/400)) #Win Probability for the Home Team.
df['elo_prob_away']=1-df['elo_prob_home'] #Win Probability for the Away Team
df['elo1_post_home']=np.where(df['win']==1,df['elo1_pre_home']+df['K']*(1-df['elo_prob_home']),df['elo1_pre_home']+df['K']*(0-df['elo_prob_home'])) #Find New ELO Rating for Home Team based on Game Result
df['elo2_post_away']=np.where(df['win']==0,df['elo2_pre_away']+df['K']*(1-df['elo_prob_away']),df['elo2_pre_away']+df['K']*(0-df['elo_prob_away'])) #Find New ELO Rating for Away Team based on Game Result

df=df[['game_id','home','away','elo1_pre_home','elo2_pre_away','home_score','away_score','elo_prob_home','elo_prob_away','elo1_post_home','elo2_post_away']]
#Find Home Games
home=df.groupby(['game_id','home','away','elo1_pre_home','elo2_pre_away','home_score','away_score','elo_prob_home','elo_prob_away','elo1_post_home','elo2_post_away']).sum().reset_index()
home.rename(columns={'home':'team','away':'opp','elo1_pre_home':'elo_pre_team','elo2_pre_away':'elo_pre_opp','elo_prob_home':'elo_prob_team','elo_prob_away':'elo_prob_opp','elo1_post_home':'elo_post_team','elo2_post_away':'elo_post_opp','home_score':'pts_for','away_score':'pts_agn'},inplace=True)
home['win']=np.where(home['pts_for']>home['pts_agn'],1,0) #Flag Variable for Game Result. 1 for Win and 0 for Loss
home['home']=1 #Flag Variable to identify that the team in question is playing at home

#Find Away Games
away=df.groupby(['game_id','home','away','elo1_pre_home','elo2_pre_away','home_score','away_score','elo_prob_home','elo_prob_away','elo1_post_home','elo2_post_away']).sum().reset_index()
away.rename(columns={'home':'opp','away':'team','elo1_pre_home':'elo_pre_opp','elo2_pre_away':'elo_pre_team','elo_prob_home':'elo_prob_opp','elo_prob_away':'elo_prob_team','elo1_post_home':'elo_post_opp','elo2_post_away':'elo_post_team','home_score':'pts_agn','away_score':'pts_for'},inplace=True)
away['win']=np.where(away['pts_for']>away['pts_agn'],1,0) #Flag Variable for Game Result. 1 for Win and 0 for Loss
away['home']=0 #Flag Variable to identify that the team in question is playing away

final=pd.concat([home,away]).reset_index() #Concatenate the Home and Away Data Frames
final.drop(['index'],axis=1,inplace=True)
final['elo_win']=np.where(final['elo_prob_team']>final['elo_prob_opp'],1,0) #A Game Result is a Win if the Probability of a Team winning the game is greater than that of the Opposition.
gsw=final.loc[final['team']=='GSW'].sort_values(by=['game_id']) #Identify only Golden State Warriors Matches and sort them in ascending order of game id
gsw_elo=[1529.010908] #Create a list of ELO Ratings and initalize the list with GSW's ELO Rating prior to the start of the first game
l=list(gsw['elo_post_team']) #Convert the ELO Ratings of GSW at the end of every game into a list named l
for i in l:
  gsw_elo.append(i) #Append the ELO Ratings at the end of every game to the gsw_elo list

gsw_elo_roundedup=[round(num) for num in gsw_elo] #Round off the values in this list to the nearest whole number

gsw_dict={'gsw_game_id':list(range(0,len(gsw_elo))) , 'ELO_Rating':gsw_elo_roundedup} #Create a Dictionary with 2 keys. The first key holds a new set of game ids starting from zero to indicate ascending order of only GSW's games
#The second key contains the ELO Ratings of GSW for the corresponding gameids

gsw_df=pd.DataFrame(data=gsw_dict) #Convert Dictionary to a Data Frame

#Graph Out the Variation of GSW's ELO Rating
#Consider the ELO Rating at Game ID 0 to be GSW's initial ELO rating at the beginning of the season prior to playing any game
import plotly.express as px


fig = px.line(gsw_df, x="gsw_game_id", y="ELO_Rating",markers=True)
fig.update_traces(textposition="bottom right")
fig.update_layout(title='Variation of ELO Rating of the Golden State Warriors over the Course of the Season',
                   xaxis_title='Games',
                   yaxis_title='ELO Rating')
fig.show()

In [34]:
# To plot a graph illustrating GSW's ranking in the league by win/loss record, we need to:
# 1. Calculate the win/loss record for all teams.
# 2. Rank the teams by their win/loss record.
# 3. Track GSW's rank over the course of the season.

# Let's start by loading the NBA ELO data and preparing the win/loss record for all teams.

import pandas as pd

# Load the NBA ELO data
elo_df = pd.read_csv('nba_elo_latest.csv')

# Filter the dataset for the 2022 season.
elo_2022_df = elo_df[elo_df['season'] == 2022]

# Initialize a DataFrame to keep track of wins, losses, and win percentage for all teams.
teams = elo_2022_df['home'].unique()
team_stats = pd.DataFrame(index=teams, columns=['Wins', 'Losses', 'WinPct'])

# Initially set Wins and Losses to 0.
team_stats['Wins'] = 0
team_stats['Losses'] = 0

# Process each game and update the wins and losses.
for i, row in elo_2022_df.iterrows():
    # Determine the winner and loser of the game.
    if row['home_score'] > row['away_score']:
        winner, loser = row['home'], row['away']
    else:
        winner, loser = row['away'], row['home']
    
    # Update the team_stats DataFrame.
    team_stats.loc[winner, 'Wins'] += 1
    team_stats.loc[loser, 'Losses'] += 1

# Calculate the win percentage for each team.
team_stats['WinPct'] = team_stats['Wins'] / (team_stats['Wins'] + team_stats['Losses'])

# Sort the teams by win percentage to get the rankings.
team_stats_sorted = team_stats.sort_values(by='WinPct', ascending=False).reset_index()
team_stats_sorted['Rank'] = range(1, len(team_stats_sorted) + 1)

# Extract GSW's rankings over time.
# For this, we need to calculate the rankings after each game, which requires iterating over each game date.
gsw_rankings_over_time = []
dates = elo_2022_df['date'].sort_values().unique()

for date in dates:
    # Filter the games up to the current date.
    games_up_to_date = elo_2022_df[elo_2022_df['date'] <= date]
    
    # Update team_stats for each team up to the current date.
    team_stats_up_to_date = team_stats.copy()
    for team in teams:
        team_games = games_up_to_date[(games_up_to_date['home'] == team) | (games_up_to_date['away'] == team)]
        wins = team_games.apply(lambda x: x['home'] == team and x['home_score'] > x['away_score'] or
                                            x['away'] == team and x['away_score'] > x['home_score'], axis=1).sum()
        losses = len(team_games) - wins
        team_stats_up_to_date.loc[team, 'Wins'] = wins
        team_stats_up_to_date.loc[team, 'Losses'] = losses
        team_stats_up_to_date.loc[team, 'WinPct'] = wins / (wins + losses) if (wins + losses) > 0 else 0
    
    # Sort the teams by win percentage to get the rankings up to the current date.
    team_stats_up_to_date_sorted = team_stats_up_to_date.sort_values(by='WinPct', ascending=False).reset_index()
    team_stats_up_to_date_sorted['Rank'] = range(1, len(team_stats_up_to_date_sorted) + 1)
    
    # Get GSW's rank on the current date and append it to the list.
    gsw_rank = team_stats_up_to_date_sorted[team_stats_up_to_date_sorted['index'] == 'GSW']['Rank'].values[0]
    gsw_rankings_over_time.append((date, gsw_rank))

# Convert the rankings over time to a DataFrame for plotting.
gsw_rankings_df = pd.DataFrame(gsw_rankings_over_time, columns=['Date', 'Rank'])

# Now let's plot the graph using Plotly.
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the ranking trend line for GSW
fig.add_trace(
    go.Scatter(
        x=gsw_rankings_df['Date'],
        y=gsw_rankings_df['Rank'],
        mode='lines+markers',
        name='GSW League Ranking'
    )
)

# Update the layout to add titles and axis labels
fig.update_layout(
    title='Golden State Warriors League Ranking Over Time',
    xaxis_title='Date',
    yaxis_title='League Ranking',
    yaxis_autorange='reversed',  # Reverse the y-axis so that a lower rank is higher on the graph.
    legend_title='Trends'
)

# Display the graph
fig.show()
